In [3]:
import pandas as pd
import numpy as np

# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)

In [5]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers']

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes']

In [10]:
# function to conditionally highlight rows based on product
def highlight_product(s,product,colour = 'yellow'):
    r = pd.Series(data = False,index = s.index)
    r['Product'] = s.loc['Product'] == product
    
    return [f'background-color: {colour}' if r.any() else '' for v in r]

# apply the formatting
df.style\
.apply(highlight_product,product = 'A',colour = '#DDEBF7', axis = 1)\
.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    }
).hide_index()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"2,288,098","452,749","£1,154,800",A,£2.55,19.79%
Jan 2012,"404,007","83,682","£663,082",B,£7.92,20.71%
Feb 2012,"1,468,180","318,702","£1,068,323",A,£3.35,21.71%
Feb 2012,"169,389","33,454","£566,514",B,£16.93,19.75%
Mar 2012,"1,239,527","488,505","£1,158,370",A,£2.37,39.41%
Mar 2012,"113,930","47,044","£615,735",B,£13.09,41.29%
Apr 2012,"1,962,100","338,577","£1,046,107",A,£3.09,17.26%
Apr 2012,"443,977","48,348","£668,153",B,£13.82,10.89%
May 2012,"1,575,188","448,902","£928,867",A,£2.07,28.50%
May 2012,"743,670","13,657","£647,340",B,£47.40,1.84%


In [11]:
# create a total "row" - i.e. column total
total = df.sum()
total['Month'] = pd.NaT
total['Product'] = ''
total['Average sale'] = total['Amounts'] / total['Numbers']
total['Product conversion'] = total['Numbers'] / total['Quotes']
total = total.to_frame().transpose()

<ipython-input-11-3a3f14feeae4>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  total = df.sum()


In [12]:
# function to highlight the total row
def highlight_total(s):
    r = pd.Series(data = False,index = s.index)
    r['Month'] = pd.isnull(s.loc['Month'])
    
    return ['font-weight: bold' if r.any() else '' for v in r]

In [13]:
# stack and reset index
d = pd.concat([df,total],axis = 0)
d.reset_index(drop = True,inplace = True)

# apply formatting
d.style\
.set_properties(**{'text-align':'center'})\
.apply(highlight_product,product = 'A',colour = '#DDEBF7',axis = 1)\
.apply(highlight_total,axis = 1)\
.format(
    {
        'Month':'{:%b %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    },
    na_rep = 'Total'
)\
.set_caption('Sales data <br> Produced by Team X')\
.hide_index()

Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
Jan 2012,"2,288,098","452,749","£1,154,800",A,£2.55,19.79%
Jan 2012,"404,007","83,682","£663,082",B,£7.92,20.71%
Feb 2012,"1,468,180","318,702","£1,068,323",A,£3.35,21.71%
Feb 2012,"169,389","33,454","£566,514",B,£16.93,19.75%
Mar 2012,"1,239,527","488,505","£1,158,370",A,£2.37,39.41%
Mar 2012,"113,930","47,044","£615,735",B,£13.09,41.29%
Apr 2012,"1,962,100","338,577","£1,046,107",A,£3.09,17.26%
Apr 2012,"443,977","48,348","£668,153",B,£13.82,10.89%
May 2012,"1,575,188","448,902","£928,867",A,£2.07,28.50%
May 2012,"743,670","13,657","£647,340",B,£47.40,1.84%
